In [ ]:
import geopandas as gpd
from shapely import geometry


In [ ]:
data_file = "/Users/alex/Data/bathymetry/fiji_4326.gpkg"
data = gpd.read_file(data_file)
data

In [ ]:
ll = (-17.77804,177.39404)
ur = (-17.71414,177.44241)

subset = data.cx[ll[1]:ur[1], ll[0]:ur[0]]

In [ ]:
data_utm = subset.to_crs(epsg=32760)

# Get the extent of the shapefile
total_bounds = data_utm.total_bounds

# Get minX, minY, maxX, maxY
minX, minY, maxX, maxY = total_bounds

# Create a fishnet
x, y = (minX, minY)
geom_array = []

# Polygon Size
square_size = 10
while y <= maxY:
    while x <= maxX:
        geom = geometry.Polygon([(x,y), (x, y+square_size), (x+square_size, y+square_size), (x+square_size, y), (x, y)])
        geom_array.append(geom)
        x += square_size
    x = minX
    y += square_size

fishnet = gpd.GeoDataFrame(geom_array, columns=['geometry']).set_crs(epsg=32760)

In [61]:
# Combine the fishnet with the bathymetry and get the average depth
joined = fishnet.sjoin(data_utm, how="left", predicate="intersects")
non_null = joined[joined['depth'].notna()]
# joined
grouped = non_null.groupby('index_right')

# Get the average depth and the geometry
average_depth = grouped['depth'].mean()
average_depth = average_depth.reset_index()
average_depth = average_depth.rename(columns={"depth": "average_depth"})
average_depth = average_depth.merge(data_utm, left_on='index_right', right_index=True)
average_depth = average_depth.drop(columns=['index_right', 'average_depth'])

# Convert to geopandas frame
average_depth = gpd.GeoDataFrame(average_depth, crs=32760)


In [62]:
average_depth.to_file("random_forest_regressor/nadi.geojson")